# About Dataset
Context

"Predict behavior to retain customers. You can analyze all relevant customer data and develop focused customer retention programs." [IBM Sample Data Sets]

Content
Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

The data set includes information about:

Customers who left within the last month – the column is called Churn
Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
Demographic info about customers – gender, age range, and if they have partners and dependents

# Giới Thiệu Tập Dữ Liệu (Vietsub)
Ngữ Cảnh

"Dự đoán hành vi để giữ chân khách hàng. Bạn có thể phân tích tất cả dữ liệu liên quan đến khách hàng và phát triển các chương trình giữ chân khách hàng tập trung." [Tập Dữ Liệu Mẫu của IBM]

Nội Dung

Mỗi hàng đại diện cho một khách hàng, mỗi cột chứa các thuộc tính của khách hàng được mô tả trong phần Metadata của cột.

Tập dữ liệu bao gồm thông tin về:

Khách hàng đã rời bỏ trong tháng qua – cột này được gọi là Churn
Dịch vụ mà mỗi khách hàng đã đăng ký – bao gồm điện thoại, nhiều đường dây, internet, bảo mật trực tuyến, sao lưu trực tuyến, bảo vệ thiết bị, hỗ trợ kỹ thuật, và truyền hình và phim trực tuyến
Thông tin tài khoản khách hàng – thời gian khách hàng đã là khách hàng, hợp đồng, phương thức thanh toán, hóa đơn điện tử, phí hàng tháng, và tổng phí
Thông tin nhân khẩu học về khách hàng – giới tính, độ tuổi, và liệu họ có đối tác và người phụ thuộc hay không

# Load Libraries

In [36]:
import pandas as pd
import xgboost as xgb 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
#Tải dữ liệu vào
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
#Xem thông tin và kiểu dữ liệu các bảng
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


# Cleansing Data

In [8]:
#Chuyển đổi dữ liệu cột TotalCharges về dạng integer
df['TotalCharges'] = pd.to_numeric(df['MonthlyCharges'], errors='coerce')

In [10]:
#Xem các trường bị Null
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [12]:
#Xem tổng cột và dòng
df.shape

(7043, 21)

In [15]:
#Xóa cột CustomerID trong bảng df
df = df.drop(['customerID'], axis=1)
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,56.95,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,53.85,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,42.30,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,70.70,Yes


In [19]:
#Tiến hành đưa dữ liệu đầu vào bằng hai mục X, y
X = df.drop(['Churn'], axis=1)
y = df['Churn'].copy()

In ra hai tệp KQ hiển thị như sau:

In [22]:
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,56.95
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,53.85
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,42.30
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,70.70


In [23]:
y.head()

0     No
1     No
2    Yes
3     No
4    Yes
Name: Churn, dtype: object

In [26]:
#Chuyển các cột có trường Object về dạng số trong phương thức  one-hot encoding
X_ec = pd.get_dummies(X, drop_first=True)

In [27]:
X_ec.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,29.85,29.85,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
1,0,34,56.95,56.95,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0,2,53.85,53.85,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
3,0,45,42.30,42.30,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,0,2,70.70,70.70,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0


In [28]:
#Chuyển các cột của y về dạng số trong phương thức  one-hot encoding
y_ec = pd.get_dummies(y, drop_first=True)

In [29]:
y_ec.head()

,Yes
0,0
1,0
2,1
3,0
4,1


In [46]:
#Tính tổng KH rời bỏ/số KH rời bỏ dịch vụ
Ratio_CustomerChurn = y_ec.sum()/len(y_ec)
Ratio_CustomerChurn

Yes    0.26537
dtype: float64

In [51]:
#Đưa dữ liệu vào các tập huấn luyện train & test
X_train, X_test, y_train, y_test = train_test_split(X_ec, y_ec, random_state=42, stratify = y_ec)

In [53]:
#Tính tổng KH rời bỏ/số KH rời bỏ dịch vụ
Ratio_CustomerChurn = y_train.sum()/len(y_train)

# Convert the result to a float if it's a single-element Series
if isinstance(Ratio_CustomerChurn, pd.Series):
    Ratio_CustomerChurn = Ratio_CustomerChurn.item()
    
# Print the ratio with two decimal places
print(f'Ratio of Customer Churn: {Ratio_CustomerChurn:.2f}')

Ratio of Customer Churn: 0.27


In [56]:
# Định nghĩa mô hình
cdf_xgb = xgb.XGBClassifier(
    objective='binary:logistic',
    missing=None,
    seed=42
)

# Cài đặt các tham số bằng phương thức set_params
cdf_xgb.set_params(
    eval_metric='aucpr',
    early_stopping_rounds=10
)

# Huấn luyện mô hình
cdf_xgb.fit(X_train, y_train, 
            eval_set=[(X_test, y_test)], 
            verbose=2)

[0]	validation_0-aucpr:0.62693
[2]	validation_0-aucpr:0.64239
[4]	validation_0-aucpr:0.63778
[6]	validation_0-aucpr:0.64378
[8]	validation_0-aucpr:0.64215
[10]	validation_0-aucpr:0.64922
[12]	validation_0-aucpr:0.65056
[14]	validation_0-aucpr:0.64860
[16]	validation_0-aucpr:0.64892
[18]	validation_0-aucpr:0.65070
[20]	validation_0-aucpr:0.64508


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=None, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Kết luận: 

Hiệu Suất Của Mô Hình:

Các giá trị aucpr của mô hình đã đạt được trong các vòng lặp huấn luyện là khá tốt. Bạn có thể thấy rằng giá trị aucpr thay đổi theo từng vòng lặp và có xu hướng ổn định hoặc tăng dần trong quá trình huấn luyện.
Mức cao nhất của aucpr là 0.65070, đạt được ở vòng lặp số 18.

Chọn Mô Hình Tốt Nhất:

Thực Hành Tốt Nhất: Để chọn mô hình tốt nhất, bạn nên sử dụng giá trị aucpr cao nhất trong quá trình huấn luyện. Theo kết quả của bạn, mô hình ở vòng lặp số 18 là lựa chọn tốt nhất với aucpr = 0.65070.
Kiểm Tra Trên Dữ Liệu Thực Tế: Để xác thực, bạn cũng nên kiểm tra mô hình này trên dữ liệu kiểm tra (hoặc dữ liệu chưa thấy) để đánh giá hiệu suất thực tế.

Sự Đánh Giá Kết Quả:

Giá Trị Tốt Nhất: Trường hợp tốt nhất trong tập kết quả này là vòng lặp số 18 với aucpr = 0.65070. Đây là giá trị cao nhất bạn đã đạt được và có thể là điểm dừng tốt cho mô hình.
Sự Ổn Định: Mặc dù có một số biến động trong các giá trị của aucpr, mô hình có xu hướng duy trì giá trị ở mức ổn định sau vòng lặp số 10.

# Tiến hành Grid Search để tối ưu mô hình XGBoost

In [58]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Định nghĩa mô hình
model = xgb.XGBClassifier(objective='binary:logistic', seed=42)

# Định nghĩa các tham số để tìm kiếm
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'n_estimators': [50, 100, 150],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Khởi tạo GridSearchCV với scoring='roc_auc'
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=3, verbose=2)

# Huấn luyện với GridSearchCV
grid_search.fit(X_train, y_train)

# Lấy tham số tốt nhất
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Huấn luyện mô hình với các tham số tốt nhất
best_model = grid_search.best_estimator_

Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsampl

[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=9, n_estimators=150, subsample=0.8; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=9, n_estimators=150, subsample=0.8; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=9, n_estimators=150, subsample=0.8; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=9, n_estimators=150, subsample=0.9; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=9, n_estimators=150, subsample=0.9; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=9, n_estimators=150, subsample=0.9; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=9, n_estimators=150, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=9, n_estimators=150, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=9, n_estima

[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=100, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=100, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, n_estimators=100,

[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=9, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=9, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=9, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=9, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=9, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=9, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=9, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=9, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.2, max_depth=9, n_estimators=100, subsamp

[CV] END colsample_bytree=0.9, learning_rate=0.01, max_depth=6, n_estimators=150, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.9, learning_rate=0.01, max_depth=6, n_estimators=150, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.9, learning_rate=0.01, max_depth=6, n_estimators=150, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.9, learning_rate=0.01, max_depth=6, n_estimators=150, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.9, learning_rate=0.01, max_depth=6, n_estimators=150, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.9, learning_rate=0.01, max_depth=6, n_estimators=150, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.9, learning_rate=0.01, max_depth=6, n_estimators=150, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.9, learning_rate=0.01, max_depth=6, n_estimators=150, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.9, learning_rate=0.01, max_depth=9, n_estima

...

[CV] END colsample_bytree=0.9, learning_rate=0.2, max_depth=6, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.9, learning_rate=0.2, max_depth=6, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.9, learning_rate=0.2, max_depth=6, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.9, learning_rate=0.2, max_depth=6, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.9, learning_rate=0.2, max_depth=6, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.9, learning_rate=0.2, max_depth=6, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.9, learning_rate=0.2, max_depth=6, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=0.9, learning_rate=0.2, max_depth=6, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.9, learning_rate=0.2, max_depth=6, n_estimators=100, subsam

[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=3, n_estimators=150, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=3, n_estimators=150, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=3, n_estimators=150, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=3, n_estimators=150, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=3, n_estimators=150, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=3, n_estimators=150, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=3, n_estimators=150, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=6, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.01, max_depth=6, n_estimat

[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=150, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=150, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.1, max_depth=3, n_estimators=150,

[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=3, n_estimators=50, subsample=1.0; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=3, n_estimators=100, sub

[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=9, n_estimators=150, subsample=0.9; total time=   0.3s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=9, n_estimators=150, subsample=1.0; total time=   0.3s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=9, n_estimators=150, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=1.0, learning_rate=0.2, max_depth=9, n_estimators=150, subsample=1.0; total time=   0.3s
Best parameters found:  {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}


--> Best parameters found:  {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}